In [ ]:
#Executar caso não tenha instalado o módulo pybdedata
!pip install pybdedata

In [1]:
import requests as req
from requests.models import PreparedRequest
import pandas as pd
import json
import pybdedata.query as bde
import ipywidgets as widgets

In [2]:
fonteDatas = {
    'IBGE':'https://apisidra.ibge.gov.br/values',
    'TSE':'https://sig.tse.jus.br/ords/dwtse/f?p=2001:101',
    'INEP':{
        'Superior':'https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_',
        'Básica':'https://download.inep.gov.br/dados_abertos/sinopses_estatisticas/sinopses_estatisticas_censo_escolar_'
    },
}

In [3]:
def formatUnitMilhar(valor):
        try: 
            teste = float(valor)
            numero = str(valor).split('.')
            if len(numero) > 1:
                decimal = ','+numero[1]
            else:
                decimal = ''    
            valor = numero[0]
            if len(valor) <= 3:
                valor = valor
            elif (len(valor) > 3) and len(valor) < 7:
                valor = valor[:-3]+'.'+valor[-3:]
            elif (len(valor) > 6) and  len(valor) < 10:
                valor = valor[:-6]+'.'+valor[-6:-3]+'.'+valor[-3:]
            elif (len(valor) > 9) and  len(valor) < 13:
                valor = valor[:-9]+'.'+valor[-9:-6]+'.'+valor[-6:-3]+'.'+valor[-3:]
            elif (len(valor) > 12) and len(valor) < 17:
                valor = valor[:-12]+'.'+valor[-12:-9]+'.'+valor[-9:-6]+'.'+valor[-6:-3]+'.'+valor[-3:]
            valor = valor+decimal
        except:
            valor = valor
        return valor 
    
def getSource(pesquisa):
     
    #Dados utilizado no BDE-GO
    fontes = {
              'PEVS':{
                  '289':{'colunas':['D1C','D2C','D3N','D4N','V'],'parametros':{'v':'144,145','c193':'all'}}, #Produção da Extração Vegetal e da Silvicultura
                  '291':{'colunas':['D1C','D2C','D3N','D4N','V'],'parametros':{'v':'142,143','c194':'all'}}#Produção da Extração Vegetal e da Silvicultura
               },
                  
              'PPM':{
                '74':{'colunas':['D1C','D2C','D3C','D4C','V'],'parametros':{'v':'106','c80':'all'}},
                '94':{'colunas':['D1C','D2C','D3C','V'],'parametros':{'v':'107'}},
                '3939':{'colunas':['D1C','D2C','D3C','D4C','V'],'parametros':{'v':'105','c79':'all'}} 
              },
              'PAM':{
                  '839':{'colunas':['D1C','D2C','D3C','D4N','V'],'parametros':{'v':'214,216','c81':'allxt'}}, 
                  '1002':{'colunas':['D1C','D2C','D3C','D4N','V'],'parametros':{'v':'214,216','c81':'allxt'}},
                  '5457':{'colunas':['D1C','D2C','D3C','D4N','V'],'parametros':{'v':'214,216','c782':'allxt'}}
              }
              #'1378':{'colunas':['D2C','D4C','D3N','V'],'parametros':{'v':'93','c1':'all','c2':'all','c287':'all','c455':'all'}}, #Censo Demográfico
              #'2759':{'colunas':['D1C','D7N','V'],'parametros':{'v':'221','c236':'0','c247':'0','c248':'0','c245':'0','c246':'0'}}, #Pesquisa Estatísticas do Registro Civil
              #'5938':{'colunas':['D1C','D2N','V'],'parametros':{'v':'37,498,513,517,6575,525'}}, #Produto Interno Bruto dos Municípios
              #'6579':{'colunas':['D1C','D2N','V'],'parametros':{'v':'9324'}} #Estimativas de População
    }
    
    tbComplete = pd.DataFrame()
    pesq = fontes[pesquisa]
    
    for cod in pesq.keys():
                
        dados = pesq[cod]['parametros']#fontes[cod]['parametros']
        colunas = pesq[cod]['colunas']#fontes[cod]['colunas']
        
        #Configurando os parametros para acessar a API do IBGE
        param = {'t':cod}
        for value in dados.keys():
            param[value] = dados[value]
        param['n3'] = '52'
        param['n6'] = 'in n3 52'    
        print(param)   
        #Preparando a url para extrair os dados    
        url = PreparedRequest()
        url.prepare_url('https://apisidra.ibge.gov.br/values',param) 
        url = str(url.url).replace('&','/').replace('=','/').replace('?','/').replace('+',' ')
            
        try:
            #Extraindo os dados
            data = req.get(url)
            data.encoding = 'utf-8' # Garante a leitura correta de caracteres especiais
            api_data = json.loads(data.text)
            
            # A primeira linha da API é o cabeçalho
            if not api_data or len(api_data) < 2:
                continue
            dfdata = pd.DataFrame(data=api_data[1:], columns=api_data[0])
                        
            #Transformação dos dados
            dfdata = dfdata[colunas]
            dfdata.rename(columns = {
                'D3N':'loc_nome', # Coluna de Município (Nome)
                'D4N':'ano' # Coluna de Ano
            },inplace = True)
            dfdata['codFonte'] = dfdata[colunas[:-2]].apply(lambda x: '-'.join(x.astype(str)),axis=1)
            dfdata['loc_cod_ibge'] = dfdata['loc_nome'].str.extract(r'(\d{7})', expand=False)
            dfdata = dfdata[['loc_cod_ibge','codFonte','ano','V', 'loc_nome']]
        
            #Atualizando a tabela principal
            tbComplete = pd.concat([tbComplete, dfdata])
        except:
            continue
    
        
    #Retornar a tabela estruturda com os dados
    return tbComplete

def transformData(df):
    #Transformação dos dados
    dfdata = dfdata[colunas]
    dfdata.rename(columns = {colunas[-2]:'loc_nome'},inplace = True)
    dfdata['codFonte'] = dfdata[colunas[:-2]].apply(lambda x: '-'.join(x.values),axis=1)
    dfdata = dfdata.drop(dfdata.index[0])
    dfdata = dfdata[['loc_nome','codFonte','Ano','V']]
    return 

In [4]:
#'Raspando' os dados do IBGE
pesquisa = 'PEVS'
dados = getSource(pesquisa)
dados

{'t': '289', 'v': '144,145', 'c193': 'all', 'n3': '52', 'n6': 'in n3 52'}
{'t': '291', 'v': '142,143', 'c194': 'all', 'n3': '52', 'n6': 'in n3 52'}


,loc_cod_ibge,codFonte,ano,V
1,0,144,Goiás,..
2,0,144,Abadia de Goiás - GO,..
3,0,144,Abadiânia - GO,..
4,0,144,Acreúna - GO,..
5,0,144,Adelândia - GO,..
...,...,...,...,...
6332,3463,143,Urutaí - GO,-
6333,3463,143,Varjão - GO,-
6334,3463,143,Vianópolis - GO,-
6335,3463,143,Vicentinópolis - GO,-


In [5]:
len(dados.codFonte.unique())

4

In [6]:
#Acessando a tabela da relação das variáveis do BDE com o IBGE
src = r'Z:/GEDE/BDE - Banco de Dados/tb_bde-cod-ibge.xlsx'
dfvar = pd.read_excel(src)
dfvar = dfvar[dfvar['codFonte'].notnull()]
dfvar['codFonte'] = dfvar['codFonte'].apply(lambda x: str(x))
dfvar

,var_cod,codFonte,varNome
0,1,93-0-0-0-0,População Censitária - Total
1,2,93-1-0-0-0,População Censitária - Urbana
2,3,93-2-0-0-0,População Censitária - Rural
3,4,93-0-4-0-0,População Censitária - Masculina
4,5,93-0-5-0-0,População Censitária - Feminina
...,...,...,...
433,503,142-3457,Silvicultura - Madeira em Tora - Quantidade
434,504,143-3457,Silvicultura - Madeira em Tora - Valor
435,505,143-0,Silvicultura - Total - Valor
699,829,144-39409,Extração Vegetal - Pequi - Quantidade


In [7]:
dfvar.codFonte.unique()

array(['93-0-0-0-0', '93-1-0-0-0', '93-2-0-0-0', '93-0-4-0-0',
       '93-0-5-0-0', '93-1-4-0-0', '93-1-5-0-0', '93-2-4-0-0',
       '93-2-5-0-0', '2047-0', '2047-4', '2047-5', '2047-1', '2047-2',
       '9324', '105-2670', '105-32794', '107', '216-40129', '214-40129',
       '216-40092', '214-40092', '216-40099', '214-40099', '216-40100',
       '214-40100', '216-40101', '214-40101', '216-40102', '214-40102',
       '216-40136', '214-40136', '216-40104', '214-40104', '216-40105',
       '214-40105', '216-40137', '214-40137', '216-40139', '214-40139',
       '216-40106', '214-40106', '216-40107', '214-40107', '216-40109',
       '214-40109', '216-40145', '214-40145', '216-40110', '214-40110',
       '216-40112', '214-40112', '216-117991', '214-117991', '216-117992',
       '214-117992', '216-117993', '214-117993', '216-40148', '214-40148',
       '216-40113', '214-40113', '216-40114', '214-40114', '216-40149',
       '214-40149', '216-40151', '214-40151', '216-40152', '214-40152',
    

In [8]:
bdeObj = bde.BDEquery()
locations = pd.DataFrame(data=bdeObj.getLocations)
newrows = pd.DataFrame(data={'loc_cod':[1000],'loc_nome':['ESTADO DE GOIÁS'],'loc_cod_ibge':['52']})
locations = pd.concat([locations,newrows])
locations.reset_index(inplace=True)
locations['loc_cod_ibge'] = locations['loc_cod_ibge'].astype(str) # Garante que o tipo da coluna seja o mesmo para o merge
locations


,index,loc_cod,loc_nome,loc_cod_ibge
0,0,8,Abadia de Goiás,5200050
1,1,9,Abadiânia,5200100
2,2,10,Acreúna,5200134
3,3,11,Adelândia,5200159
4,4,12,Água Fria de Goiás,5200175
...,...,...,...,...
242,242,243,Vianópolis,5222005
243,243,244,Vicentinópolis,5222054
244,244,245,Vila Boa,5222203
245,245,246,Vila Propício,5222302


In [9]:
dados = dados.merge(locations,on='loc_cod_ibge')
dados

,loc_cod_ibge,codFonte,ano,V,index,loc_cod,loc_nome


In [10]:
dados.loc_cod_ibge.unique()

array([], dtype=object)

In [11]:
#Transformação dos dados (Unir e Criação de novos atributos)
datamerge = dados.merge(dfvar,on='codFonte')
datamerge['Valor'] = datamerge['V'].apply(lambda x: formatUnitMilhar(x))
#datamerge = datamerge.merge(dfvar,on='codFonte')
datamerge

,loc_cod_ibge,codFonte,ano,V,index,loc_cod,loc_nome,var_cod,varNome,Valor


In [12]:
#Quantidade de variáveis
len(datamerge.var_cod.unique())

0

In [ ]:
db = datamerge.copy()
fieldValue = 'd_'+str(db.ano.unique()[0])
db.rename(columns={'Valor':fieldValue},inplace=True)
db = db[['loc_nome','loc_cod','var_cod',fieldValue]]
db['loc_nome'] = db['loc_nome'].str.upper()
db


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
db.var_cod.unique()

In [ ]:
#Verificar valores nulos na base de dados
datamerge.isnull().sum()

In [ ]:
#Verificar se todos os municipios foram preenchidos pela quantidade de variaveis
report = {}
for j in datamerge.loc_nome.unique():
    data = datamerge[datamerge.loc_nome==j]
    qtd = len(data)
    report[j] = qtd

report = sorted(report.items(), key=lambda x:x[1],reverse=True)
report

In [ ]:
#exporta para csv
db.to_csv('CAMINHO DO ARQUIVO', encoding='utf-8-sig', index=False)